<a href="https://colab.research.google.com/github/sumojit/GLA-Code-Base/blob/master/External_data_Local_Files%2C_Drive%2C_Sheets%2C_and_Cloud_Storage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook provides recipes for loading and saving data from external sources.

# Local file system

## Uploading files from your local file system

`files.upload` returns a dictionary of the files which were uploaded.
The dictionary is keyed by the file name and values are the data which were uploaded.

In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

## Downloading files to your local file system

`files.download` will invoke a browser download of the file to your local computer.


In [ ]:
from google.colab import files

with open('example.txt', 'w') as f:
  f.write('some content')

files.download('example.txt')

# Google Drive

You can access files in Drive in a number of ways, including:
- Mounting your Google Drive in the runtime's virtual machine
- Using a wrapper around the API such as [PyDrive2](https://docs.iterative.ai/PyDrive2/)
- Using the [native REST API](https://developers.google.com/drive/v3/web/about-sdk)



Examples of each are below.

## Mounting Google Drive locally

The example below shows how to mount your Google Drive on your runtime using an authorization code, and how to write and read files there. Once executed, you will be able to see the new file (`foo.txt`) at [https://drive.google.com/](https://drive.google.com/).

This only supports reading, writing, and moving files; to programmatically modify sharing settings or other metadata, use one of the other options below.

**Note:** When using the 'Mount Drive' button in the file browser, no authentication codes are necessary for notebooks that have only been edited by the current user.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code
Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
with open('/content/drive/My Drive/foo.txt', 'w') as f:
  f.write('Hello Google Drive!')
!cat /content/drive/My\ Drive/foo.txt

Hello Google Drive!

In [ ]:
drive.flush_and_unmount()
print('All changes made in this colab session should now be visible in Drive.')

All changes made in this colab session should now be visible in Drive.


## PyDrive2

The examples below demonstrate authentication and file upload/download using PyDrive2. More examples are available in the [PyDrive2 documentation](https://docs.iterative.ai/PyDrive2/).

In [ ]:
from pydrive2.auth import GoogleAuth
from pydrive2.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

Authenticate and create the PyDrive2 client.


In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

Create and upload a text file.


In [ ]:
uploaded = drive.CreateFile({'title': 'Sample upload.txt'})
uploaded.SetContentString('Sample upload file content')
uploaded.Upload()
print('Uploaded file with ID {}'.format(uploaded.get('id')))

Uploaded file with ID 14vDAdqp7BSCQnoougmgylBexIr2AQx2T


Load a file by ID and print its contents.


In [ ]:
downloaded = drive.CreateFile({'id': uploaded.get('id')})
print('Downloaded content "{}"'.format(downloaded.GetContentString()))

Downloaded content "Sample upload file content"


## Drive REST API

In order to use the Drive API, we must first authenticate and construct an API client.


In [ ]:
from google.colab import auth
auth.authenticate_user()
from googleapiclient.discovery import build
drive_service = build('drive', 'v3')

With this client, we can use any of the functions in the [Google Drive API reference](https://developers.google.com/drive/v3/reference/). Examples follow.


### Creating a new Drive file with data from Python

First, create a local file to upload.

In [ ]:
with open('/tmp/to_upload.txt', 'w') as f:
  f.write('my sample file')

print('/tmp/to_upload.txt contains:')
!cat /tmp/to_upload.txt

/tmp/to_upload.txt contains:
my sample file

Upload it using the [`files.create`](https://developers.google.com/drive/v3/reference/files/create) method. Further details on uploading files are available in the [developer documentation](https://developers.google.com/drive/v3/web/manage-uploads).

In [ ]:
from googleapiclient.http import MediaFileUpload

file_metadata = {
  'name': 'Sample file',
  'mimeType': 'text/plain'
}
media = MediaFileUpload('/tmp/to_upload.txt',
                        mimetype='text/plain',
                        resumable=True)
created = drive_service.files().create(body=file_metadata,
                                       media_body=media,
                                       fields='id').execute()
print('File ID: {}'.format(created.get('id')))

File ID: 1Cw9CqiyU6zbXFD9ViPZu_3yX-sYF4W17


After executing the cell above, you will see a new file named 'Sample file' at [https://drive.google.com/](https://drive.google.com/).

### Downloading data from a Drive file into Python

Download the file we uploaded above.

In [ ]:
file_id = created.get('id')

import io
from googleapiclient.http import MediaIoBaseDownload

request = drive_service.files().get_media(fileId=file_id)
downloaded = io.BytesIO()
downloader = MediaIoBaseDownload(downloaded, request)
done = False
while done is False:
  # _ is a placeholder for a progress object that we ignore.
  # (Our file is small, so we skip reporting progress.)
  _, done = downloader.next_chunk()

downloaded.seek(0)
print('Downloaded file contents are: {}'.format(downloaded.read()))

Downloaded file contents are: b'my sample file'


In order to download a different file, set `file_id` above to the ID of that file, which will look like "1uBtlaggVyWshwcyP6kEI-y_W3P8D26sz".

# Google Sheets


## Google Sheets Workspace Extension

We have a Workspace Extension, [Sheets to Colab](https://workspace.google.com/u/0/marketplace/app/sheets_to_colab/945625412720), which allows you to directly import data from Google Sheets into Colab from the Sheets UI. Follow the link to the Sheets to Colab Workspace Extension to learn more.

## Interacting with Google Sheets using gspread

 You can also use the open-source [`gspread`](https://github.com/burnash/gspread) library to interact with Google Sheets. The code below shows you how to setup and authenticate `gspread`.

In [ ]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

Below is a small set of `gspread` examples. Additional examples are available at the [`gspread` GitHub page](https://github.com/burnash/gspread#more-examples).

### Creating a new sheet with data from Python

In [1]:
!pip install pyspark py4j

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.2-py2.py3-none-any.whl size=317812365 sha256=367e95a8bc6d2af5284dee72643198e6b759091d94b6df38bf7da706d6881458
  Stored in directory: /root/.cache/pip/wheels/34/34/bd/03944534c44b677cd5859f248090daa9fb27b3c8f8e5f49574
Successfully built pyspark


In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import desc, asc,column,count,max,when

spark=SparkSession.builder.appName("test_pyspark").getOrCreate()


# create two dataframes
df1 = spark.createDataFrame([(1, "John", 25,2500), (2, "Jane", 30,3000), (3, "Jim", 35,4000)],
                           ["id", "name", "age","salary"])
df1.show()
# second highest salary....sql
df1.createOrReplaceTempView("SalaryView")
df2=spark.sql("select * from (select * from SalaryView order by salary desc limit 2) order by salary asc limit 1").show()


#second highest salary---------dataframe..

df3=df1.select("id","name","age","salary").orderBy(column("salary").desc()).limit(2)
# Calculate the count and collect the result as a scalar value
record_count = df3.count()
# Use lit to create a column from the boolean condition
df4 = df3.withColumn("RecordCnt", when(lit(record_count > 1), 1).otherwise(None)) # Add a new column with the count if it's greater than 1
df4.show()
df5=df3.select("id","name","age","salary").orderBy(column("salary").asc()).limit(1)
df5.show()

+---+----+---+------+
| id|name|age|salary|
+---+----+---+------+
|  1|John| 25|  2500|
|  2|Jane| 30|  3000|
|  3| Jim| 35|  4000|
+---+----+---+------+

+---+----+---+------+
| id|name|age|salary|
+---+----+---+------+
|  2|Jane| 30|  3000|
+---+----+---+------+



NameError: name 'lit' is not defined

In [10]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import desc, asc,column,count,max,when,lit,length

spark=SparkSession.builder.appName("test_pyspark").getOrCreate()


# create two dataframes
df1 = spark.createDataFrame([(1, "John", 25,2500), (2, "Jane", 30,3000), (3, "Jim", 35,4000)],
                           ["id", "name", "age","salary"])
df1.show()
# second highest salary....sql
df1.createOrReplaceTempView("SalaryView")
df2=spark.sql("select * from (select * from SalaryView order by salary desc limit 2) order by salary asc limit 1")


#second highest salary---------dataframe..

df3=df1.select("id","name","age","salary").orderBy(column("salary").desc()).limit(2)
# Calculate the count and collect the result as a scalar value
record_count = df3.select("id").count()
if  record_count>1:
  df3.select("id","name","age","salary").orderBy(column("salary").asc()).limit(1).show()
else:
  print("Null")


#nth highest salary ---
employees_Salary = [("James", "Sales", 2000),
("sofy", "Sales", 3000),
("Laren", "Sales", 4000),
("Kiku", "Sales", 5000),
("Sam", "Finance", 6000),
("Samuel", "Finance", 7000),
("Yash", "Finance", 8000),
("Rabin", "Finance", 9000),
("Lukasz", "Marketing", 10000),
("Jolly", "Marketing", 11000),
("Mausam", "Marketing", 12000),
("Lamba", "Marketing", 13000),
("Jogesh", "HR", 14000),
("Mannu", "HR", 15000),
("Sylvia", "HR", 16000),
("Sama", "HR", 17000),
]

employeesDF = spark.createDataFrame(employees_Salary,schema="""employee_name STRING, dept_name STRING, salary INTEGER""")
employeesDF.select("employee_name","dept_name","salary",length("salary")).show()



+---+----+---+------+
| id|name|age|salary|
+---+----+---+------+
|  1|John| 25|  2500|
|  2|Jane| 30|  3000|
|  3| Jim| 35|  4000|
+---+----+---+------+

+---+----+---+------+
| id|name|age|salary|
+---+----+---+------+
|  2|Jane| 30|  3000|
+---+----+---+------+

+-------------+---------+------+--------------+
|employee_name|dept_name|salary|length(salary)|
+-------------+---------+------+--------------+
|        James|    Sales|  2000|             4|
|         sofy|    Sales|  3000|             4|
|        Laren|    Sales|  4000|             4|
|         Kiku|    Sales|  5000|             4|
|          Sam|  Finance|  6000|             4|
|       Samuel|  Finance|  7000|             4|
|         Yash|  Finance|  8000|             4|
|        Rabin|  Finance|  9000|             4|
|       Lukasz|Marketing| 10000|             5|
|        Jolly|Marketing| 11000|             5|
|       Mausam|Marketing| 12000|             5|
|        Lamba|Marketing| 13000|             5|
|       Jogesh

In [16]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import desc, asc,column,count,max,when,lit,length

spark=SparkSession.builder.appName("nthHighestSalary").getOrCreate()
#nth highest salary ---
employees_Salary = [("James", "Sales", 2000),
("sofy", "Sales", 3000),
("Laren", "Sales", 4000),
("Kiku", "Sales", 5000),
("Sam", "Finance", 6000),
("Samuel", "Finance", 7000),
("Yash", "Finance", 8000),
("Rabin", "Finance", 9000),
("Lukasz", "Marketing", 10000),
("Jolly", "Marketing", 11000),
("Mausam", "Marketing", 12000),
("Lamba", "Marketing", 13000),
("Jogesh", "HR", 14000),
("Mannu", "HR", 15000),
("Sylvia", "HR", 16000),
("Sama", "HR", 17000),
]

employeesDF = spark.createDataFrame(employees_Salary,schema="""employee_name STRING, dept_name STRING, salary INTEGER""")


# nth lowest salary
#libraries for window and rank
from pyspark.sql.window import Window
from pyspark.sql.functions import rank,dense_rank

#Create window specification for applying window function
windowPartition = Window.partitionBy("dept_name").orderBy("salary")

#Apply the window specification
employeeDF = employeesDF.withColumn("rank", rank().over(windowPartition))
employeeDF.filter("rank=1").show()


#nth highest salary
WindowPartition=Window.partitionBy(employeesDF.dept_name).orderBy(desc(employeesDF.salary))
employeeDF=employeesDF.withColumn("rank",dense_rank().over(WindowPartition))
employeeDF.show()
employeeDF.filter("rank=1").show()

+-------------+---------+------+----+
|employee_name|dept_name|salary|rank|
+-------------+---------+------+----+
|          Sam|  Finance|  6000|   1|
|       Jogesh|       HR| 14000|   1|
|       Lukasz|Marketing| 10000|   1|
|        James|    Sales|  2000|   1|
+-------------+---------+------+----+

+-------------+---------+------+----+
|employee_name|dept_name|salary|rank|
+-------------+---------+------+----+
|        Rabin|  Finance|  9000|   1|
|         Yash|  Finance|  8000|   2|
|       Samuel|  Finance|  7000|   3|
|          Sam|  Finance|  6000|   4|
|         Sama|       HR| 17000|   1|
|       Sylvia|       HR| 16000|   2|
|        Mannu|       HR| 15000|   3|
|       Jogesh|       HR| 14000|   4|
|        Lamba|Marketing| 13000|   1|
|       Mausam|Marketing| 12000|   2|
|        Jolly|Marketing| 11000|   3|
|       Lukasz|Marketing| 10000|   4|
|         Kiku|    Sales|  5000|   1|
|        Laren|    Sales|  4000|   2|
|         sofy|    Sales|  3000|   3|
|        Ja